# Classification of MNIST digits with dense layers

## Loading the image data

In [0]:
from keras.datasets import mnist
import numpy as np

(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

Using TensorFlow backend.


## Preparing the image data

In [0]:
train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

## Preparing the labels

We need to categorically encode the labels.

In [0]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)

## Calculating Width

In [0]:
def width_calculation(image_array):
  images_copy = np.copy(image_array)
  for i in range(28):
    [1 if j >= 128 else 0 for j in images_copy[i]]
    
  ymin = 29
  ymax = -1

  for i in range(28):
    for j in range(28):
      if images_copy[i][j] == 1 and j < ymin:
        ymin = j
      elif images_copy[i][j] == 1 and j > ymax:
        ymax = j
  width = (ymax-ymin + 1)/28

  return width

width_array_train_images = np.zeros((60000))
width_array_test_images = np.zeros((10000))
for i in range(60000):
  print("index", i)
  width_array_train_images[i] = width_calculation(train_images_original[i])

for i in range(10000):
  print("index", i)
  width_array_test_images = width_calculation(test_images_original[i])


Calculate height

In [0]:
def height_calculation(image_array):
  images_copy = np.copy(image_array)
  for i in range(28):
    [1 if j >= 128 else 0 for j in images_copy[i]]
    
  xmin = 29
  xmax = -1

  for i in range(28):
    for j in range(28):
      if images_copy[i][j] == 1 and j < xmin:
        xmin = j
      elif images_copy[i][j] == 1 and j > xmax:
        xmax = j
  height = (xmax - xmin + 1)/28

  return height

height_array_train_images = np.zeros((60000))
height_array_test_images = np.zeros((10000))
for i in range(60000):
  print("this is index", i)
  height_array_train_images[i] = height_calculation(train_images_original[i])

for i in range(10000):
  print("this is index", i)
  height_array_test_images = height_calculation(test_images_original[i])

Connected Regions

In [0]:
next_value = 2

def floodWhiteRegion(image_array, row, col, current_value):
  global next_value
  if (row >= 28) or (col >= 28):
    return
  if (row < 0) or (col < 0):
    return
  if image_array[row][col] == 1:
    return
  if image_array[row][col] > 1:
    return
  
  if image_array[row][col] == 0:
    image_array[row][col] = current_value
  
  floodWhiteRegion(image_array, row - 1, col, current_value)
  floodWhiteRegion(image_array, row + 1, col, current_value)
  floodWhiteRegion(image_array, row, col - 1, current_value)
  floodWhiteRegion(image_array, row, col + 1, current_value)

  next_value = current_value + 1

def count_connected_regions(image_array):
  global next_value
  image_array_copy = np.copy(image_array)
  for i in range(28):
    [1 if j >= 128 else 0 for j in image_array_copy[i]]
  next_value = 2

  for i in range(28):
    for j in range(28):
      floodWhiteRegion(image_array_copy, i, j, nextVal)

  return next_value - 2

connected_regions_array_train_images = np.zeros((60000))
connected_regions_array_test_images = np.zeros((10000))
for i in range(60000):
  print(i)
  connected_regions_array_train_images[i] = count_connected_regions(train_images_original[i])

for i in range(10000):
  connected_regions_array_test_images[i] = count_connected_regions(test_images_original[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128


KeyboardInterrupt: ignored

In [0]:
train_images = np.column_stack((train_images, width_array_train_images))
train_images = np.column_stack((train_images, height_array_train_images))
train_images = np.column_stack((train_images, connected_regions_array_train_images))

test_images = np.column_stack((test_images, width_array_test_images))
test_images = np.column_stack((test_images, height_array_test_images))
test_images = np.column_stack((test_images, connected_regions_array_test_images))


## The network architecture

In [0]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(787,)))
network.add(layers.Dense(10, activation='softmax'))
network.summary()

## The compilation step

In [0]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

## The training step

In [0]:
epochs = 10
history = network.fit(train_images, 
                      train_labels, 
                      epochs=epochs, 
                      batch_size=128, 
                      validation_data=(test_images, test_labels))

## Display the training/test loss/accuracy

In [0]:
history_dict = history.history
loss_values = history_dict['loss']
test_loss_values = history_dict['val_loss']
epochs_range = range(1, epochs + 1)

In [0]:
plt.plot(epochs_range, loss_values, 'bo', label='Training loss')
plt.plot(epochs_range, test_loss_values, 'ro', label='Test loss')
plt.title('Training and test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
acc_values = history_dict['acc']
test_acc_values = history_dict['val_acc']

In [0]:
plt.plot(epochs_range, acc_values, 'bo', label='Training accuracy')
plt.plot(epochs_range, test_acc_values, 'ro', label='Test accuracy')
plt.title('Training and test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Showing some misclassified digits

In [0]:
predictions = network.predict(test_images)

In [0]:
predictions[0]

In [0]:
np.argmax(predictions[0])

In [0]:
test_labels_original[0]

In [0]:
predicted_labels = np.argmax(predictions, axis=1) 
predicted_labels = predicted_labels.astype('uint8')

In [0]:
predicted_labels

In [0]:
wrong_indices = [i for i in range(10000) if predicted_labels[i] != test_labels_original[i]]

In [0]:
# The quantity below is equal to the test accuracy reported by the network fit method.
1.0 - len(wrong_indices) / 10000.0

In [0]:
# This show the ith misclassified digit. 
i = 4
wrong_index = wrong_indices[i]  

In [0]:
predicted_labels[wrong_index]

In [0]:
test_labels_original[wrong_index]

In [0]:
plt.imshow(test_images_original[wrong_index])
plt.show()